In [1]:
import sys
sys.path.append('/Users/isaaclin/Desktop/lee-lab-research/whole_m1s1_analysis')
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
from torch.utils.data import Dataset,DataLoader
from modeling.full_sacnn import *
from modeling.reduced_sascnn import *
from modeling.scnn import *
from utils import *
import os

vimg = np.load('../all_sites_data_prepared/pics_data/val_img_m1s1.npy')
vresp = np.load('../all_sites_data_prepared/New_response_data/valRsp_m1s1.npy')
vimg = np.reshape(vimg,(1000,1,50,50))

In [2]:
net0a = net_one_neuron_orig()
net2 = net_one_neuron_fuller(5,5,5)
net3 = net_one_neuron_fuller(30,30,30)
net16 = net_one_neuron_reduced(5,5,5,False)
net17 = net_one_neuron_reduced(30,30,30,False)

count_parameters(net0a)
count_parameters(net2)
count_parameters(net3)
count_parameters(net16)
count_parameters(net17)

+------------------+------------+
|     Modules      | Parameters |
+------------------+------------+
| layers.0.weight  |    750     |
|  layers.0.bias   |     30     |
| layers.2.weight  |     30     |
|  layers.2.bias   |     30     |
| layers.5.weight  |   22500    |
|  layers.5.bias   |     30     |
| layers.7.weight  |     30     |
|  layers.7.bias   |     30     |
| layers.10.weight |    8100    |
|  layers.10.bias  |     30     |
| layers.11.weight |     30     |
|  layers.11.bias  |     30     |
| layers.14.weight |    8100    |
|  layers.14.bias  |     30     |
| layers.15.weight |     30     |
|  layers.15.bias  |     30     |
|  Linear.weight   |    750     |
|   Linear.bias    |     1      |
+------------------+------------+
Total Trainable Params: 40561
+-----------------------------+------------+
|           Modules           | Parameters |
+-----------------------------+------------+
|      layers_1.0.weight      |    750     |
|       layers_1.0.bias       |     30    

29222

# m0a_results analysis

In [ ]:
rootdir = "./m0a_results/baseline_scnn_m0a/"

to_plot = [] #(neuron num, pc, plot)

for subdir, dirs, files in os.walk(rootdir):
    if 'model' not in files[0] or subdir[-3:] == 'avg':
        continue
    print(subdir,dirs,files)

    neuron_number = int(subdir.split("_")[-1])

    files.sort(key = lambda x: int(x.split(".")[0].split("_")[-1]),reverse=True)
    print(files)
    net = net_one_neuron_orig()
    net.load_state_dict(torch.load(subdir + "/" + files[0], map_location=torch.device('cpu')))

    net_resp = torch.from_numpy(np.float32(vimg))
    with torch.no_grad():
        net.eval()
        net_resp = net(net_resp).detach().numpy()
    net_resp = np.transpose(net_resp)[0]

    real_resp = vresp[:,neuron_number]
    real_resp = np.transpose(real_resp)

    comb_resp_ord = []
    for r,n in zip(real_resp,net_resp):
        comb_resp_ord.append((r,n))
    comb_resp_ord.sort(key = lambda x:x[0],reverse=True)
    to_plot.append((neuron_number, pc_one_neuron(net,neuron_number,vimg,vresp), comb_resp_ord))

to_plot.sort(key= lambda x: x[0])
np.save("to_plot_m0a",to_plot)

In [ ]:
figure, axis = plt.subplots(9, 9, figsize=(50,50))
for neuron_num,pc,tplot in to_plot:
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[0] for x in tplot])
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[1] for x in tplot],linewidth = '.5')
    axis[neuron_num//9,neuron_num%9].set_title(f'NN: {neuron_num}, PC = {round(pc,3)}')
    axis[neuron_num//9,neuron_num%9].set_ylabel("PC")

plt.savefig("m0a_tc")
plt.show()


# m2_results analysis

In [ ]:
rootdir = "./m2_results/full_sascnn_m2/"

to_plot = [] #(neuron num, pc, plot)

for subdir, dirs, files in os.walk(rootdir):
    if 'model' not in files[0] or subdir[-3:] == 'avg':
        continue
    print(subdir,dirs,files)

    neuron_number = int(subdir.split("_")[-1])

    files.sort(key = lambda x: int(x.split(".")[0].split("_")[-1]),reverse=True)
    print(files)
    net = net_one_neuron_fuller(q_dim=5,k_dim=5,v_dim=5)
    net.load_state_dict(torch.load(subdir + "/" + files[0], map_location=torch.device('cpu')))

    net_resp = torch.from_numpy(np.float32(vimg))
    with torch.no_grad():
        net.eval()
        net_resp = net(net_resp).detach().numpy()
    net_resp = np.transpose(net_resp)[0]

    real_resp = vresp[:,neuron_number]
    real_resp = np.transpose(real_resp)

    comb_resp_ord = []
    for r,n in zip(real_resp,net_resp):
        comb_resp_ord.append((r,n))
    comb_resp_ord.sort(key = lambda x:x[0],reverse=True)
    to_plot.append((neuron_number, pc_one_neuron(net,neuron_number,vimg,vresp), comb_resp_ord))

to_plot.sort(key= lambda x: x[0])
np.save("to_plot_m16",to_plot)

In [ ]:
figure, axis = plt.subplots(9, 9, figsize=(50,50))
for neuron_num,pc,tplot in to_plot:
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[0] for x in tplot])
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[1] for x in tplot],linewidth = '.5')
    axis[neuron_num//9,neuron_num%9].set_title(f'NN: {neuron_num}, PC = {round(pc,3)}')
    axis[neuron_num//9,neuron_num%9].set_ylabel("PC")

plt.savefig("m16_tc")
plt.show()


# m3_results analysis

In [ ]:
rootdir = "./m3_results/full_sascnn_m3/"

to_plot = [] #(neuron num, pc, plot)

for subdir, dirs, files in os.walk(rootdir):
    if 'model' not in files[0] or subdir[-3:] == 'avg':
        continue
    print(subdir,dirs,files)

    neuron_number = int(subdir.split("_")[-1])

    files.sort(key = lambda x: int(x.split(".")[0].split("_")[-1]),reverse=True)
    print(files)
    net = net_one_neuron_fuller(q_dim=30,k_dim=30,v_dim=30)
    net.load_state_dict(torch.load(subdir + "/" + files[0], map_location=torch.device('cpu')))

    net_resp = torch.from_numpy(np.float32(vimg))
    with torch.no_grad():
        net.eval()
        net_resp = net(net_resp).detach().numpy()
    net_resp = np.transpose(net_resp)[0]

    real_resp = vresp[:,neuron_number]
    real_resp = np.transpose(real_resp)

    comb_resp_ord = []
    for r,n in zip(real_resp,net_resp):
        comb_resp_ord.append((r,n))
    comb_resp_ord.sort(key = lambda x:x[0],reverse=True)
    to_plot.append((neuron_number, pc_one_neuron(net,neuron_number,vimg,vresp), comb_resp_ord))

to_plot.sort(key= lambda x: x[0])
np.save("to_plot_m3",to_plot)

In [ ]:
figure, axis = plt.subplots(9, 9, figsize=(50,50))
for neuron_num,pc,tplot in to_plot:
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[0] for x in tplot])
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[1] for x in tplot],linewidth = '.5')
    axis[neuron_num//9,neuron_num%9].set_title(f'NN: {neuron_num}, PC = {round(pc,3)}')
    axis[neuron_num//9,neuron_num%9].set_ylabel("PC")

plt.savefig("m3_tc")
plt.show()


# m16_results analysis

In [ ]:
rootdir = "./m16_results/reduced_sascnn_m16/"

to_plot = [] #(neuron num, pc, plot)

for subdir, dirs, files in os.walk(rootdir):
    if 'model' not in files[0] or subdir[-3:] == 'avg':
        continue
    print(subdir,dirs,files)

    neuron_number = int(subdir.split("_")[-1])

    files.sort(key = lambda x: int(x.split(".")[0].split("_")[-1]),reverse=True)
    print(files)
    net = net_one_neuron_reduced(q_dim=5,k_dim=5,v_dim=5,v_mapping=False)
    net.load_state_dict(torch.load(subdir + "/" + files[0], map_location=torch.device('cpu')))

    net_resp = torch.from_numpy(np.float32(vimg))
    with torch.no_grad():
        net.eval()
        net_resp = net(net_resp).detach().numpy()
    net_resp = np.transpose(net_resp)[0]

    real_resp = vresp[:,neuron_number]
    real_resp = np.transpose(real_resp)

    comb_resp_ord = []
    for r,n in zip(real_resp,net_resp):
        comb_resp_ord.append((r,n))
    comb_resp_ord.sort(key = lambda x:x[0],reverse=True)
    to_plot.append((neuron_number, pc_one_neuron(net,neuron_number,vimg,vresp), comb_resp_ord))

to_plot.sort(key= lambda x: x[0])
np.save("to_plot_m16",to_plot)

In [ ]:
figure, axis = plt.subplots(9, 9, figsize=(50,50))
for neuron_num,pc,tplot in to_plot:
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[0] for x in tplot])
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[1] for x in tplot],linewidth = '.5')
    axis[neuron_num//9,neuron_num%9].set_title(f'NN: {neuron_num}, PC = {round(pc,3)}')
    axis[neuron_num//9,neuron_num%9].set_ylabel("PC")

plt.savefig("m16_tc")
plt.show()

# m17_results analysis

In [ ]:
rootdir = "./m17_results/reduced_sascnn_m17/"

to_plot = [] #(neuron num, pc, plot)

for subdir, dirs, files in os.walk(rootdir):
    if 'model' not in files[0] or subdir[-3:] == 'avg':
        continue
    print(subdir,dirs,files)

    neuron_number = int(subdir.split("_")[-1])

    files.sort(key = lambda x: int(x.split(".")[0].split("_")[-1]),reverse=True)
    print(files)
    net = net_one_neuron_reduced(q_dim=30,k_dim=30,v_dim=30,v_mapping=False)
    net.load_state_dict(torch.load(subdir + "/" + files[0], map_location=torch.device('cpu')))

    net_resp = torch.from_numpy(np.float32(vimg))
    with torch.no_grad():
        net.eval()
        net_resp = net(net_resp).detach().numpy()
    net_resp = np.transpose(net_resp)[0]

    real_resp = vresp[:,neuron_number]
    real_resp = np.transpose(real_resp)

    comb_resp_ord = []
    for r,n in zip(real_resp,net_resp):
        comb_resp_ord.append((r,n))
    comb_resp_ord.sort(key = lambda x:x[0],reverse=True)
    to_plot.append((neuron_number, pc_one_neuron(net,neuron_number,vimg,vresp), comb_resp_ord))

to_plot.sort(key= lambda x: x[0])
np.save("to_plot_m17",to_plot)

In [ ]:
figure, axis = plt.subplots(9, 9, figsize=(50,50))
for neuron_num,pc,tplot in to_plot:
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[0] for x in tplot])
    axis[neuron_num//9,neuron_num%9].plot([i for i in range(1000)],[x[1] for x in tplot],linewidth = '.5')
    axis[neuron_num//9,neuron_num%9].set_title(f'NN: {neuron_num}, PC = {round(pc,3)}')
    axis[neuron_num//9,neuron_num%9].set_ylabel("PC")

plt.savefig("m17_tc")
plt.show()

# Correlation Comparisons

In [ ]:
to_plot_m0a = np.asarray(np.load("./to_plot_m0a.npy", allow_pickle=True))
to_plot_m2 = np.asarray(np.load("./to_plot_m2.npy", allow_pickle=True))
to_plot_m3 = np.asarray(np.load("./to_plot_m3.npy", allow_pickle=True))
to_plot_m16 = np.asarray(np.load("./to_plot_m16.npy", allow_pickle=True))
to_plot_m17= np.asarray(np.load("./to_plot_m17.npy", allow_pickle=True))

In [ ]:
labels = ["m0a","m2", "m3", "m16", "m17"]
all_corrs = np.asarray([to_plot_m0a[:,1:2], to_plot_m2[:,1:2], to_plot_m3[:,1:2], to_plot_m16[:,1:2], to_plot_m17[:,1:2]])
# print(all_corrs.shape) #(5,80,2)
per_imp = np.asarray([(all_corrs[i] - all_corrs[0])/all_corrs[0] for i in range(len(labels))])

In [40]:
np.save("all_corrs",all_corrs)
np.save("per_imp",per_imp)

In [47]:
per_imp
for i in range(len(labels)):
    print(f'Average Improvement of {labels[i]} WRT to m0a: {round(np.mean(per_imp[i])*100,3)}')

Average Improvement of m0a WRT to m0a: 0.0
Average Improvement of m2 WRT to m0a: 5.58
Average Improvement of m3 WRT to m0a: 5.823
Average Improvement of m16 WRT to m0a: 6.154
Average Improvement of m17 WRT to m0a: 6.785
